In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [2]:
'''#import necessary libraries
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import plotly.express as px'''


'#import necessary libraries\nfrom transformers import BertTokenizer, TFBertModel\nimport matplotlib.pyplot as plt\nimport plotly.express as px'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Sentence Correlation/train.xlsx")
evaluation = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Sentence Correlation/evaluation.xlsx")

In [5]:
train

,text,reason,label
0,this is an amazing app for online classes!but,good app for conducting online classes,1.0
1,very practical and easy to use,app is user-friendly,1.0
2,this app is very good for video conferencing.,good for video conferencing,1.0
3,i can not download this zoom app,unable to download zoom app,1.0
4,i am not able to download this app,want to download the app,1.0
...,...,...,...
2056,i am not getting jio tv properly.,unable to use with jio tv,1.0
2057,(learn from the netflix interface),netflix ui is better,1.0
2058,"hello,\ndisney+ must also be installed on chro...",unable to use disney on chromecast,1.0
2059,it is a shame that disney+ does not work on tv...,unable to use disney plus on tv,1.0


In [6]:
#set up TPU
'''
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

    '''

"\ntry:\n    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()\n    tf.config.experimental_connect_to_cluster(tpu)\n    tf.tpu.experimental.initialize_tpu_system(tpu)\n    strategy = tf.distribute.experimental.TPUStrategy(tpu)\nexcept ValueError:\n    strategy = tf.distribute.get_strategy() # for CPU and single GPU\n    print('Number of replicas:', strategy.num_replicas_in_sync)\n\n    "

In [7]:
#load the data
'''train=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
train.head()'''

'train=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")\ntrain.head()'

In [ ]:
%%capture
!pip install transformers

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("activebus/BERT-XD_Review")
model = AutoModel.from_pretrained("activebus/BERT-XD_Review")

Some weights of the model checkpoint at activebus/BERT-XD_Review were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Starting with a pre-trained model

model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

"\nmodel_name = 'bert-base-multilingual-cased'\ntokenizer = BertTokenizer.from_pretrained(model_name)"

In [ ]:
#function to encode string to numbers
def encode_sen(s):
    tokens=list(tokenizer.tokenize(s))
    tokens.append('[SEP]')
    return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
encode_sen("The rules developed in the interim")

[1996, 3513, 2764, 1999, 1996, 9455, 102]

In [ ]:
#Encoding the input variables

def bert_encode(hypotheses, premises, tokenizer):
    num_examples=len(hypotheses)

    sen1=tf.ragged.constant([encode_sen(s)
        for s in np.array(hypotheses)])
    sen2=tf.ragged.constant([encode_sen(s)
        for s in np.array(premises)])

    cls=[tokenizer.convert_tokens_to_ids(['[CLS]'])]*sen1.shape[0]
    input_word_ids=tf.concat([cls,sen1,sen2], axis=-1)

    input_mask=tf.ones_like(input_word_ids).to_tensor()

    type_cls=tf.zeros_like(cls)
    type_s1=tf.zeros_like(sen1)
    type_s2=tf.ones_like(sen2)
    input_type_ids=tf.concat([type_cls, type_s1, type_s2], axis=-1).to_tensor()

    inputs={
    'input_word_ids':input_word_ids.to_tensor(),
    'input_mask':input_mask,
    'input_type_ids':input_type_ids}

    return inputs

In [ ]:
#train_input=bert_encode(train.hypothesis.values,train.premise.values,tokenizer)

train_input=bert_encode(train.text.values, train.reason.values,tokenizer)

In [ ]:
#Creating and training model

max_len = 50

def build_model(model):
    bert_encoder = TFBertModel.from_pretrained(model_name)
    input_word_ids=tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask=tf.keras.Input(shape=(max_len,), dtype=tf.int32, name='input_mask')
    input_type_ids=tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
    
    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
if True:#with strategy.scope():
    model = build_model(model)
    model.summary()

In [ ]:
model.fit(train_input, train.label.values, epochs = 2, verbose = 1, batch_size = 64, validation_split = 0.2)

In [ ]:
#Predict the lables for test

test_input=bert_encode(test.hypothesis.values,test.premise.values,tokenizer)
predictions = [np.argmax(i) for i in model.predict(test_input)]

In [ ]:
submission=test['id'].to_frame()

In [ ]:
submission['prediction']=predictions

In [ ]:
submission.to_csv("submission.csv", index=False)